In [1]:
// Usual setup: importing Semantic Kernel SDK and SkiaSharp, used to display images inline.
#r "nuget: StackExchange.Redis"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: Microsoft.SemanticKernel, 0.16.230615.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Redis, 0.16.230615.1-preview"

Installed Packages Microsoft.Extensions.Logging.Console, 7.0.0 Microsoft.SemanticKernel, 0.16.230615.1-preview Microsoft.SemanticKernel.Connectors.Memory.Redis, 0.16.230615.1-preview StackExchange.Redis, 2.6.116

In [2]:
#!import config/Settings.cs
#!import config/Utils.cs

In [3]:
using StackExchange.Redis;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.Memory.Redis;

In [4]:
ConnectionMultiplexer connectionMultiplexer = await ConnectionMultiplexer.ConnectAsync("localhost:6379");
IDatabase database = connectionMultiplexer.GetDatabase();
RedisMemoryStore memoryStore = new RedisMemoryStore(database, vectorSize: 1536);

In [5]:
ILogger logger = NullLogger.Instance;

In [6]:
// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Configure text embedding and chat completion services with memmory storage.
var kernel = new KernelBuilder()
    .WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey)
    .WithOpenAIChatCompletionService("gpt-4", apiKey, orgId)
    .WithMemoryStorage(memoryStore)
    .WithLogger(logger)
    .Build();

In [7]:
// Get AI service instance used to manage the user chat
var chatGPT = kernel.GetService<IChatCompletion>();

In [8]:
const string MemoryCollectionName = "aboutMe";

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Richard");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a Software Engineer");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Pretoria and have been living there since 1997");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I have recently visited the Neatherlands and will be going to Keyan soon.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is also from Pretoria");

In [9]:
using System.Linq;
using System.Collections.Generic;

var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var responses = kernel.Memory.SearchAsync(MemoryCollectionName, q);

    await foreach (var response in responses)
    {
         Console.WriteLine(q + " " + response?.Metadata.Text);
    }
}

what is my name? My name is Richard
where do I live? I currently live in Pretoria and have been living there since 1997
where is my family from? My family is also from Pretoria
where have I travelled? I have recently visited the Neatherlands and will be going to Keyan soon.
what do I do for work? I currently work as a Software Engineer


In [10]:
using Microsoft.SemanticKernel.CoreSkills;

// TextMemorySkill provides the "recall" function
kernel.ImportSkill(new TextMemorySkill());

In [11]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

Information about me, from previous conversations:
- {{$fact1}} {{recall $fact1}}
- {{$fact2}} {{recall $fact2}}
- {{$fact3}} {{recall $fact3}}
- {{$fact4}} {{recall $fact4}}
- {{$fact5}} {{recall $fact5}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var chatFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0.8);

In [12]:
var context = kernel.CreateNewContext();

context["fact1"] = "what is my name?";
context["fact2"] = "where do I live?";
context["fact3"] = "where is my family from?";
context["fact4"] = "where have I travelled?";
context["fact5"] = "what do I do for work?";

context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.8";

In [13]:
var history = "";
context["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nChatBot: {answer}\n";
    context["history"] = history;
    
    // Show the bot response
    Console.WriteLine("ChatBot: " + context);
};